In [1]:
import gym
import CustomGymEnvs
import torchgraphs as tg
import torch
import torch.nn as nn

from collections import OrderedDict

In [2]:
env = gym.make("FetchReachEnv-v0")

In [3]:
num_nodes = env.observation_space['observation_nodes'].shape[0]
num_edges = env.observation_space['observation_edges'].shape[0]

In [85]:
# g1 = tg.GraphBatch(
#     node_features=torch.FloatTensor(env.robot_graph.node_features),
#     edge_features=torch.FloatTensor(env.robot_graph.edge_features),
#     senders=torch.tensor(env.robot_graph.edges_from),
#     receivers=torch.tensor(env.robot_graph.edges_to),
#     global_features=torch.randn([1, 10]),
#     num_nodes_by_graph=torch.tensor([num_nodes]),
#     num_edges_by_graph=torch.tensor([num_edges])
# )

g1 = tg.Graph(
    node_features=torch.FloatTensor(env.robot_graph.node_features),
    edge_features=torch.FloatTensor(env.robot_graph.edge_features),
    senders=torch.tensor(env.robot_graph.edges_from),
    receivers=torch.tensor(env.robot_graph.edges_to),
    global_features=torch.randn([1, 10])
)

In [86]:
# g2 = tg.GraphBatch(
#     node_features=torch.FloatTensor(env.robot_graph.node_features),
#     edge_features=torch.FloatTensor(env.robot_graph.edge_features),
#     senders=torch.tensor(env.robot_graph.edges_from),
#     receivers=torch.tensor(env.robot_graph.edges_to),
#     global_features=torch.randn([1, 10]),
#     num_nodes_by_graph=torch.tensor([num_nodes]),
#     num_edges_by_graph=torch.tensor([num_edges])
# )

g2 = tg.Graph(
    node_features=torch.FloatTensor(env.robot_graph.node_features),
    edge_features=torch.FloatTensor(env.robot_graph.edge_features),
    senders=torch.tensor(env.robot_graph.edges_from),
    receivers=torch.tensor(env.robot_graph.edges_to),
    global_features=torch.randn([1, 10]),
)

In [6]:
dataloader = torch.utils.data.DataLoader([g1, g2], collate_fn=tg.GraphBatch.collate)

In [94]:
a = tg.GraphBatch.collate([g1, g2])

In [99]:
a.global_features = torch.randn([2, 10])

In [101]:
a.global_features.shape

torch.Size([2, 10])

In [79]:
class QNetwork(nn.Module):
    def __init__(self, num_node_features, num_edge_features, num_actions):
        # For the action value function, we consider the action as the graph's global features
        super(QNetwork, self).__init__()
        
        self.edge1 = tg.EdgeLinear(256,
                                   edge_features=num_edge_features,
                                   sender_features=num_node_features,
                                   receiver_features=num_node_features,
                                   global_features=num_actions)
        self.edge_relu1 = tg.EdgeReLU()

        self.node1 = tg.NodeLinear(256,
                                   node_features=num_node_features,
                                   incoming_features=256,
                                   global_features=num_actions,
                                   aggregation='avg')
        self.node_relu1 = tg.NodeReLU()

        self.global1 = tg.GlobalLinear(int(num_actions / 2),
                                       node_features=256,
                                       edge_features=256,
                                       global_features=num_actions,
                                       aggregation='avg')
        self.global_relu = tg.GlobalReLU()

        self.edge2 = tg.EdgeLinear(128,
                                   edge_features=256,
                                   sender_features=256,
                                   receiver_features=256,
                                   global_features=int(num_actions / 2))
        self.edge_relu2 = tg.EdgeReLU()

        self.node2 = tg.NodeLinear(128,
                                   node_features=256,
                                   incoming_features=128,
                                   global_features=int(num_actions / 2),
                                   aggregation='avg')
        self.node_relu2 = tg.NodeReLU()

        self.global_output = tg.GlobalLinear(1,
                                             node_features=128,
                                             edge_features=128,
                                             global_features=int(num_actions / 2),
                                             aggregation='avg')

    def forward(self, g):
        print(g)
        g = self.edge_relu1(self.edge1(g))
        print(g)
        g = self.node_relu1(self.node1(g))
        print(g)
        g = self.global_relu(self.global1(g))
        print(g)
        g = self.edge_relu2(self.edge2(g))
        print(g)
        g = self.node_relu2(self.node2(g))
        print(g)
        return self.global_output(g)

In [80]:
q = QNetwork(g1.node_features_shape[0], g1.edge_features_shape[0], 10)

In [83]:
# q(tg.GraphBatch.collate((g1, g2)))
q(g1).global_features

GraphBatch(#1, n=tensor([28]), e=tensor([27]), n_shape=torch.Size([14]), e_shape=torch.Size([10]), g_shape=torch.Size([10]))
GraphBatch(#1, n=tensor([28]), e=tensor([27]), n_shape=torch.Size([14]), e_shape=torch.Size([256]), g_shape=torch.Size([10]))
GraphBatch(#1, n=tensor([28]), e=tensor([27]), n_shape=torch.Size([256]), e_shape=torch.Size([256]), g_shape=torch.Size([10]))
GraphBatch(#1, n=tensor([28]), e=tensor([27]), n_shape=torch.Size([256]), e_shape=torch.Size([256]), g_shape=torch.Size([5]))
GraphBatch(#1, n=tensor([28]), e=tensor([27]), n_shape=torch.Size([256]), e_shape=torch.Size([128]), g_shape=torch.Size([5]))
GraphBatch(#1, n=tensor([28]), e=tensor([27]), n_shape=torch.Size([128]), e_shape=torch.Size([128]), g_shape=torch.Size([5]))


tensor([[1.7000]], grad_fn=<AddBackward0>)

In [26]:
rb = ReplayMemory(30, 0)

for _ in range(20):
    g = tg.Graph(
        node_features=torch.tensor(env.robot_graph.node_features),
        edge_features=torch.tensor(env.robot_graph.edge_features),
        senders=torch.tensor(env.robot_graph.edges_from),
        receivers=torch.tensor(env.robot_graph.edges_to),
    )
    rb.push(g, 1, 2, g, False)

In [6]:

torch.cat((torch.rand([256, 6]), torch.rand([256, 1])), 1).shape

torch.Size([256, 7])

In [1]:
a = {'a': 1, 'b': 2, 'c': 3}

In [6]:
import numpy as np
np.array(list(a.values())) / 2

array([0.5, 1. , 1.5])